In [1]:
import torch

from model import CLIPModel
from tokenizer import CLIPTextTokenizer
from loss import InfoNCECriterion
from train import Trainer
from config import CLIPConfig
from dataset import CLIPDataset
from torch.utils.data import DataLoader


# Reload all modules



/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())

True


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = CLIPConfig()
tokenizer = CLIPTextTokenizer(config)

train_dataloader = DataLoader(
    CLIPDataset(config, 'train', tokenizer=tokenizer.tokenize_text),
    batch_size=config.batch_size, 
    shuffle=True, 
    num_workers=config.num_workers
)
val_dataloader = DataLoader(
        CLIPDataset(config, 'val', tokenizer=tokenizer.tokenize_text),
        batch_size=config.batch_size, 
        shuffle=False, 
        num_workers=config.num_workers
    )

Skipping non-image file: image
Skipping non-image file: image


In [3]:
!pip install wandb onnx -Uq

import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import model, dataset, tokenizer, loss, utils, train,config
importlib.reload(model)
importlib.reload(dataset)
importlib.reload(tokenizer)
importlib.reload(loss)
importlib.reload(utils)
importlib.reload(train)
importlib.reload(config)
from model import CLIPModel
from tokenizer import CLIPTextTokenizer
from loss import InfoNCECriterion
from train import Trainer
from config import CLIPConfig
from dataset import CLIPDataset
from torch.utils.data import DataLoader


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

import wandb


run = wandb.init(project="clip-training", entity="arpan12pallar-arcbest",reinit=True)


Using device: cuda


wandb: Currently logged in as: arpan12pallar (arpan12pallar-arcbest) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [5]:

config_ = CLIPConfig()

model = CLIPModel(config_)
criterion = InfoNCECriterion()


run.watch(model, criterion, log="all", log_freq=1)

trainer = Trainer(config_, model,train_dataloader,val_dataloader, criterion, device, tokenizer)
trainer.run()

Current training run directory created at: runs/run_2025-07-21_18-37-01
Checkpoint directory created at: runs/run_2025-07-21_18-37-01/checkpoints
Starting training...
Using device: cuda
*********  Epoch 1/2  *********
[train_epoch] Execution time: 0:02:39637939453125
Epoch 1 - Train Loss: 4.1317
Epoch 1 - Validation Loss: 0.0000
Best model saved at epoch 1 with validation loss: 0.0000
*********  Epoch 2/2  *********
[train_epoch] Execution time: 0:02:34105300903324
Epoch 2 - Train Loss: 3.4872
Epoch 2 - Validation Loss: 0.0000
********* Training complete ********
Final model saved to runs/run_2025-07-21_18-37-01/final_model.pth
Training results saved to: runs/run_2025-07-21_18-37-01/results.json
********* Training Summary *********
Total epochs: 2
Final training loss: 3.4872
Final validation loss: 0.0000
Final model: runs/run_2025-07-21_18-37-01/final_model.pth
----------
Best epoch: 1
Best validation loss: 0.0000
Best model: runs/run_2025-07-21_18-37-01/best_model.pth
[run] Execution 

Starting training...
Using device: cuda
*********  Epoch 1/2  *********
[train_epoch] Execution time: 0:02:45949626922607
Epoch 1 - Train Loss: 3.9266


KeyboardInterrupt: 

In [ ]:
run.finish()

In [ ]:
test_iter = iter(test_dataloader)


In [ ]:
images, token_ids = next(test_iter)  # first batch

import matplotlib.pyplot as plt



# Step 2: Select one image and 5 captions
image = images[0].unsqueeze(0)  # Shape: (1, 3, H, W)
captions = token_ids[:5]        # Shape: (5, T)

# Step 3: Repeat the image 5 times to pair with 5 captions
image_batch = image.repeat(5, 1, 1, 1)  # Shape: (5, 3, H, W)

# Step 4: Move to device
image_batch = image_batch.to(trainer.device)
captions = captions.to(trainer.device)

#suffle captions
captions = captions[torch.randperm(captions.size(0))]  # Shuffle captions

# Step 5: Run inference
trainer.model.eval()
with torch.no_grad():
    logits = trainer.model(image_batch, captions)  # Shape: (5, 5)

# Step 6: Print logits
print("Logits (image vs 5 captions):")
print(logits)

# Optional: Compute similarity scores (e.g., softmax)
import torch.nn.functional as F
scores = F.softmax(logits[0], dim=0)  # Similarity of image to 5 captions
print("Similarity scores:", scores)

plt.imshow(images[0].permute(1, 2, 0).cpu())
plt.axis('off')
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

# Assume `captions` is a tensor of shape (5, T)
captions_text = [tokenizer.decode(caption.tolist()) for caption in captions]

print("Captions:")
for i, caption in enumerate(captions_text):
    print(f"Caption {i+1}: {caption}")

plt.show()


